In [7]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import arcpy
import numpy


In [8]:
#under normal circumstances I probably wouldn't have read these in as csv files, but I was trying to work from a different computer

low_qual_path = r"C:\Users\KOlso\Documents\ArcGIS\Projects\5571_Final_Proj\Lines_low.csv"
med_qual_path = r"C:\Users\KOlso\Documents\ArcGIS\Projects\5571_Final_Proj\Lines_med.csv"
high_qual_path = r"C:\Users\KOlso\Documents\ArcGIS\Projects\5571_Final_Proj\Lines_high.csv"

In [9]:
low_qual_df = pd.read_csv(low_qual_path)
med_qual_df = pd.read_csv(med_qual_path)
high_qual_df = pd.read_csv(high_qual_path)

low_qual_df

,Name,OriginID,DestinationID,DestinationRank,Total_Length,Shape_Length
0,1 - 1,1,1,1,0.000000,0.000000
1,1 - 68,1,68,2,1541.157223,1170.750337
2,1 - 88,1,88,3,4681.802789,2874.576249
3,1 - 76,1,76,4,6786.820728,4031.345475
4,1 - 80,1,80,5,7730.412806,6349.349078
...,...,...,...,...,...,...
8467,100 - 47,100,47,88,42506.628601,34386.514470
8468,100 - 58,100,58,89,43217.975631,34314.910116
8469,100 - 77,100,77,90,47428.959527,40828.634117
8470,100 - 42,100,42,91,47720.827352,39035.414973


In [10]:
#Remove entries where origin and destination are the same. We don't need those trips - they are all 0

low_qual_df = low_qual_df[low_qual_df['DestinationID']!=low_qual_df['OriginID']]
med_qual_df = med_qual_df[med_qual_df['DestinationID']!=med_qual_df['OriginID']]
high_qual_df = high_qual_df[high_qual_df['DestinationID']!=high_qual_df['OriginID']]

In [11]:
#define table list
tables = [low_qual_df, med_qual_df, high_qual_df]

#define speeds
speeds_mph = [8, 12, 18]

#conversion parameter for m/s to mph
mph_to_mps = 1609.34 / 3600

#Loop through all three tables and calculate time values for each speed
for table in tables:
    for speed in speeds_mph:
        column_name = f"time_{speed}mph"  # e.g., 'time_8mph'
        table[column_name] = table['Total_Length'] / (speed * mph_to_mps)
    


high_qual_df

,Name,OriginID,DestinationID,DestinationRank,Total_Length,Shape_Length,time_8mph,time_12mph,time_18mph
1,1 - 68,1,68,2,1545.039775,1170.750337,432.020517,288.013678,192.009119
2,1 - 88,1,88,3,3390.097920,2344.603808,947.931490,631.954327,421.302884
3,1 - 80,1,80,4,8038.073200,6346.563098,2247.587794,1498.391863,998.927908
4,1 - 47,1,47,5,15120.460226,13481.948608,4227.948788,2818.632525,1879.088350
5,1 - 57,1,57,6,16887.125725,15251.065221,4721.939787,3147.959858,2098.639905
...,...,...,...,...,...,...,...,...,...
1931,99 - 65,99,65,40,65397.193277,32316.303367,18286.214830,12190.809887,8127.206591
1932,99 - 78,99,78,41,66475.501192,34304.336472,18587.728843,12391.819229,8261.212819
1933,99 - 69,99,69,42,68679.022817,35938.427557,19203.872561,12802.581708,8535.054472
1934,99 - 20,99,20,43,69430.987820,36374.956668,19414.135310,12942.756873,8628.504582


In [7]:
low_qual_df['OriginID'].unique()

array([  1,   2,   3,   5,   6,   7,   8,   9,  10,  12,  13,  14,  15,
        16,  18,  20,  21,  22,  23,  24,  25,  26,  27,  28,  29,  30,
        31,  32,  33,  34,  35,  36,  37,  38,  39,  40,  41,  42,  43,
        44,  45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,  56,
        58,  60,  61,  62,  64,  65,  66,  67,  68,  69,  70,  71,  72,
        73,  75,  76,  77,  78,  79,  80,  81,  82,  83,  84,  85,  86,
        87,  88,  89,  90,  91,  92,  93,  94,  95,  96,  97,  98,  99,
       100], dtype=int64)

In [12]:
labels = ["low-qual", "med-qual", "high-qual"]

# Add label column to each table
for table, label in zip(tables, labels):
    table['network_quality'] = label
    
for table in tables:
    print (table.head())

     Name  OriginID  DestinationID  ...   time_12mph   time_18mph  network_quality
1  1 - 68         1             68  ...   287.289924   191.526616         low-qual
2  1 - 88         1             88  ...   872.743383   581.828922         low-qual
3  1 - 76         1             76  ...  1265.143611   843.429074         low-qual
4  1 - 80         1             80  ...  1441.040328   960.693552         low-qual
5  1 - 62         1             62  ...  2141.392513  1427.595009         low-qual

[5 rows x 10 columns]
     Name  OriginID  DestinationID  ...   time_12mph   time_18mph  network_quality
1  1 - 68         1             68  ...   287.289937   191.526625         med-qual
2  1 - 88         1             88  ...   763.967485   509.311656         med-qual
3  1 - 80         1             80  ...  1441.121680   960.747787         med-qual
4  1 - 76         1             76  ...  1520.674274  1013.782850         med-qual
5  1 - 62         1             62  ...  1841.662845  1227.77523

In [9]:
for i, table in enumerate(tables):
    print(f"Table {i + 1} row count: {table.shape[0]}")


Table 1 row count: 8372
Table 2 row count: 6164
Table 3 row count: 1856


In [36]:
#group[group['time_8mph']<=1800]

,Name,OriginID,DestinationID,DestinationRank,Total_Length,Shape_Length,time_8mph,time_12mph,time_18mph,quality_label
8380,100 - 100,100,100,1,0.000000,0.000000,0.000000,0.000000,0.000000,low-qual
8381,100 - 53,100,53,2,3270.659932,2091.814254,914.534511,609.689674,406.459783,low-qual
8382,100 - 25,100,25,3,5202.277911,4218.598378,1454.649148,969.766099,646.510733,low-qual
8383,100 - 75,100,75,4,5423.310500,4858.197765,1516.453779,1010.969186,673.979457,low-qual


In [10]:
def transform(data):
    '''
    Description
    
    Args:
    
    '''
    groups = data.groupby('OriginID')
    access_score = pd.DataFrame(columns=['time_8mph', 'time_12mph', 'time_18mph', 
                                         'baseline', 'delta'])
    
    for (idx, group) in groups:
        #print(idx)
        print(f'-----OriginID {idx}-----')
        #print(group)
        print(len(group[group['time_8mph']<=1800]))
        print(len(group[group['time_12mph']<=1800]))
        print(len(group[group['time_18mph']<=1800]))
        access_score.loc[idx, 'time_8mph'] = len(group[group['time_8mph']<=1800])
        access_score.loc[idx, 'time_12mph'] = len(group[group['time_12mph']<=1800])
        access_score.loc[idx, 'time_18mph'] = len(group[group['time_18mph']<=1800])
        access_score.loc[idx, 'baseline'] = len(group)
        access_score.loc[idx, 'delta'] = access_score.loc[idx, 'time_18mph'] - access_score.loc[idx, 'time_8mph']
    return access_score

In [35]:
scores_lowq = transform(low_qual_df)

-----OriginID 1-----
2
4
8
-----OriginID 2-----
5
10
32
-----OriginID 3-----
2
7
24
-----OriginID 5-----
3
6
17
-----OriginID 6-----
2
4
9
-----OriginID 7-----
6
14
35
-----OriginID 8-----
0
0
6
-----OriginID 9-----
2
7
18
-----OriginID 10-----
4
8
27
-----OriginID 12-----
2
6
12
-----OriginID 13-----
3
6
10
-----OriginID 14-----
4
10
28
-----OriginID 15-----
4
6
12
-----OriginID 16-----
1
2
11
-----OriginID 18-----
6
13
33
-----OriginID 20-----
3
8
13
-----OriginID 21-----
6
17
28
-----OriginID 22-----
7
13
25
-----OriginID 23-----
4
8
17
-----OriginID 24-----
3
6
18
-----OriginID 25-----
9
16
28
-----OriginID 26-----
2
5
18
-----OriginID 27-----
0
8
19
-----OriginID 28-----
3
5
24
-----OriginID 29-----
7
14
30
-----OriginID 30-----
3
12
24
-----OriginID 31-----
4
8
10
-----OriginID 32-----
7
15
32
-----OriginID 33-----
0
0
1
-----OriginID 34-----
3
12
28
-----OriginID 35-----
9
15
26
-----OriginID 36-----
10
15
29
-----OriginID 37-----
6
17
34
-----OriginID 38-----
1
4
12
-----Origin

In [36]:
scores_medq = transform(med_qual_df)
med_access_score

-----OriginID 1-----
2
4
6
-----OriginID 2-----
5
9
15
-----OriginID 3-----
1
2
8
-----OriginID 4-----
2
3
9
-----OriginID 5-----
2
4
5
-----OriginID 6-----
2
4
5
-----OriginID 7-----
3
8
23
-----OriginID 9-----
0
1
8
-----OriginID 10-----
3
5
12
-----OriginID 13-----
1
6
9
-----OriginID 14-----
3
4
9
-----OriginID 15-----
1
1
1
-----OriginID 17-----
0
0
2
-----OriginID 18-----
6
8
11
-----OriginID 20-----
3
6
10
-----OriginID 22-----
4
9
16
-----OriginID 23-----
3
6
14
-----OriginID 24-----
0
0
5
-----OriginID 25-----
6
16
21
-----OriginID 26-----
1
3
6
-----OriginID 27-----
0
0
3
-----OriginID 29-----
6
9
10
-----OriginID 31-----
2
4
4
-----OriginID 32-----
6
10
18
-----OriginID 35-----
4
10
15
-----OriginID 36-----
5
9
9
-----OriginID 37-----
3
8
22
-----OriginID 38-----
1
3
5
-----OriginID 39-----
3
7
16
-----OriginID 40-----
1
5
19
-----OriginID 41-----
3
3
9
-----OriginID 43-----
2
3
11
-----OriginID 44-----
3
5
5
-----OriginID 45-----
2
6
16
-----OriginID 46-----
2
3
12
-----Ori

,time_8mph_med,time_12mph_med,time_18mph_med,baseline_med,delta_med
1,2,4,6,78,4
2,5,9,15,78,10
3,1,2,8,78,7
4,2,3,9,78,7
5,2,4,5,78,3
6,2,4,5,78,3
7,3,8,23,78,20
9,0,1,8,78,8
10,3,5,12,78,9
13,1,6,9,78,8


In [37]:
scores_highq = transform(high_qual_df)

-----OriginID 1-----
2
3
3
-----OriginID 2-----
5
9
14
-----OriginID 3-----
1
2
6
-----OriginID 4-----
2
6
7
-----OriginID 5-----
2
3
3
-----OriginID 6-----
2
4
6
-----OriginID 7-----
3
4
9
-----OriginID 10-----
3
5
7
-----OriginID 13-----
2
6
7
-----OriginID 14-----
3
4
7
-----OriginID 18-----
5
6
10
-----OriginID 20-----
3
6
9
-----OriginID 22-----
1
5
6
-----OriginID 23-----
2
5
13
-----OriginID 26-----
1
3
10
-----OriginID 29-----
6
6
7
-----OriginID 31-----
2
6
7
-----OriginID 32-----
1
6
11
-----OriginID 35-----
1
1
1
-----OriginID 39-----
1
3
7
-----OriginID 41-----
3
3
7
-----OriginID 43-----
3
5
7
-----OriginID 45-----
1
1
1
-----OriginID 46-----
2
3
8
-----OriginID 47-----
1
2
3
-----OriginID 49-----
2
2
3
-----OriginID 50-----
3
6
8
-----OriginID 51-----
5
6
10
-----OriginID 52-----
3
5
7
-----OriginID 53-----
1
1
1
-----OriginID 54-----
4
8
14
-----OriginID 55-----
0
1
3
-----OriginID 56-----
1
1
1
-----OriginID 57-----
1
2
2
-----OriginID 60-----
2
2
3
-----OriginID 61----

In [38]:
scores_lowq

,time_8mph,time_12mph,time_18mph,baseline,delta
1,2,4,8,91,6
2,5,10,32,91,27
3,2,7,24,91,22
5,3,6,17,91,14
6,2,4,9,91,7
7,6,14,35,91,29
8,0,0,6,91,6
9,2,7,18,91,16
10,4,8,27,91,23
12,2,6,12,91,10


In [39]:
scores_medq

,time_8mph,time_12mph,time_18mph,baseline,delta
1,2,4,6,78,4
2,5,9,15,78,10
3,1,2,8,78,7
4,2,3,9,78,7
5,2,4,5,78,3
6,2,4,5,78,3
7,3,8,23,78,20
9,0,1,8,78,8
10,3,5,12,78,9
13,1,6,9,78,8


In [40]:
pd.set_option("display.max_rows", 100)
scores_highq

,time_8mph,time_12mph,time_18mph,baseline,delta
1,2,3,3,5,1
2,5,9,14,42,9
3,1,2,6,42,5
4,2,6,7,42,5
5,2,3,3,3,1
6,2,4,6,42,4
7,3,4,9,42,6
10,3,5,7,42,4
13,2,6,7,42,5
14,3,4,7,42,4


In [41]:
scores_highq = scores_highq.rename(columns=lambda x: f"{x}_highq")
scores_medq = scores_medq.rename(columns=lambda x: f"{x}_medq")
scores_lowq = scores_lowq.rename(columns=lambda x: f"{x}_lowq")

scores_lowq

,time_8mph_lowq,time_12mph_lowq,time_18mph_lowq,baseline_lowq,delta_lowq
1,2,4,8,91,6
2,5,10,32,91,27
3,2,7,24,91,22
5,3,6,17,91,14
6,2,4,9,91,7
7,6,14,35,91,29
8,0,0,6,91,6
9,2,7,18,91,16
10,4,8,27,91,23
12,2,6,12,91,10


In [45]:
all_scores = pd.concat([scores_highq, scores_medq, scores_lowq], axis=1)

In [47]:
pd.set_option("display.max_rows", 100)
all_scores

,time_8mph_highq,time_12mph_highq,time_18mph_highq,baseline_highq,delta_highq,time_8mph_medq,time_12mph_medq,time_18mph_medq,baseline_medq,delta_medq,time_8mph_lowq,time_12mph_lowq,time_18mph_lowq,baseline_lowq,delta_lowq
1,2,3,3,5,1,2,4,6,78,4,2,4,8,91,6
2,5,9,14,42,9,5,9,15,78,10,5,10,32,91,27
3,1,2,6,42,5,1,2,8,78,7,2,7,24,91,22
4,2,6,7,42,5,2,3,9,78,7,NaN,NaN,NaN,NaN,NaN
5,2,3,3,3,1,2,4,5,78,3,3,6,17,91,14
6,2,4,6,42,4,2,4,5,78,3,2,4,9,91,7
7,3,4,9,42,6,3,8,23,78,20,6,14,35,91,29
10,3,5,7,42,4,3,5,12,78,9,4,8,27,91,23
13,2,6,7,42,5,1,6,9,78,8,3,6,10,91,7
14,3,4,7,42,4,3,4,9,78,6,4,10,28,91,24


In [48]:
exclude_columns = ["baseline_highq", "delta_highq", "baseline_medq", "delta_medq", "baseline_lowq", "delta_lowq"]
included_columns = [col for col in all_scores.columns if col not in exclude_columns]

all_scores["overall_range"] = all_scores[included_columns].max(axis=1) - all_scores[included_columns].min(axis=1)

all_scores

,time_8mph_highq,time_12mph_highq,time_18mph_highq,baseline_highq,delta_highq,time_8mph_medq,time_12mph_medq,time_18mph_medq,baseline_medq,delta_medq,time_8mph_lowq,time_12mph_lowq,time_18mph_lowq,baseline_lowq,delta_lowq,overall_range
1,2,3,3,5,1,2,4,6,78,4,2,4,8,91,6,6
2,5,9,14,42,9,5,9,15,78,10,5,10,32,91,27,27
3,1,2,6,42,5,1,2,8,78,7,2,7,24,91,22,23
4,2,6,7,42,5,2,3,9,78,7,NaN,NaN,NaN,NaN,NaN,7
5,2,3,3,3,1,2,4,5,78,3,3,6,17,91,14,15
6,2,4,6,42,4,2,4,5,78,3,2,4,9,91,7,7
7,3,4,9,42,6,3,8,23,78,20,6,14,35,91,29,32
10,3,5,7,42,4,3,5,12,78,9,4,8,27,91,23,24
13,2,6,7,42,5,1,6,9,78,8,3,6,10,91,7,9
14,3,4,7,42,4,3,4,9,78,6,4,10,28,91,24,25


In [53]:
#convert to csv to be joined to points

csv_path = r"C:\Users\KOlso\Documents\ArcGIS\Projects\5571_Final_Proj\all_scores.csv"

all_scores.to_csv(csv_path, index=True)

In [54]:
#bring in socres to table
arcpy.conversion.ExportTable(
    in_table=r"C:\Users\KOlso\Documents\ArcGIS\Projects\5571_Final_Proj\all_scores.csv",
    out_table=r"C:\Users\KOlso\Documents\ArcGIS\Projects\5571_Final_Proj\5571_Final_Proj.gdb\all_scores",
    where_clause="",
    use_field_alias_as_name="NOT_USE_ALIAS",
    field_mapping=r'Field1 "Field1" true true false 4 Long 0 0,First,#,C:\Users\KOlso\Documents\ArcGIS\Projects\5571_Final_Proj\all_scores.csv,Field1,-1,-1;time_8mph_highq "time_8mph_highq" true true false 4 Long 0 0,First,#,C:\Users\KOlso\Documents\ArcGIS\Projects\5571_Final_Proj\all_scores.csv,time_8mph_highq,-1,-1;time_12mph_highq "time_12mph_highq" true true false 4 Long 0 0,First,#,C:\Users\KOlso\Documents\ArcGIS\Projects\5571_Final_Proj\all_scores.csv,time_12mph_highq,-1,-1;time_18mph_highq "time_18mph_highq" true true false 4 Long 0 0,First,#,C:\Users\KOlso\Documents\ArcGIS\Projects\5571_Final_Proj\all_scores.csv,time_18mph_highq,-1,-1;baseline_highq "baseline_highq" true true false 4 Long 0 0,First,#,C:\Users\KOlso\Documents\ArcGIS\Projects\5571_Final_Proj\all_scores.csv,baseline_highq,-1,-1;delta_highq "delta_highq" true true false 4 Long 0 0,First,#,C:\Users\KOlso\Documents\ArcGIS\Projects\5571_Final_Proj\all_scores.csv,delta_highq,-1,-1;time_8mph_medq "time_8mph_medq" true true false 4 Long 0 0,First,#,C:\Users\KOlso\Documents\ArcGIS\Projects\5571_Final_Proj\all_scores.csv,time_8mph_medq,-1,-1;time_12mph_medq "time_12mph_medq" true true false 4 Long 0 0,First,#,C:\Users\KOlso\Documents\ArcGIS\Projects\5571_Final_Proj\all_scores.csv,time_12mph_medq,-1,-1;time_18mph_medq "time_18mph_medq" true true false 4 Long 0 0,First,#,C:\Users\KOlso\Documents\ArcGIS\Projects\5571_Final_Proj\all_scores.csv,time_18mph_medq,-1,-1;baseline_medq "baseline_medq" true true false 4 Long 0 0,First,#,C:\Users\KOlso\Documents\ArcGIS\Projects\5571_Final_Proj\all_scores.csv,baseline_medq,-1,-1;delta_medq "delta_medq" true true false 4 Long 0 0,First,#,C:\Users\KOlso\Documents\ArcGIS\Projects\5571_Final_Proj\all_scores.csv,delta_medq,-1,-1;time_8mph_lowq "time_8mph_lowq" true true false 4 Long 0 0,First,#,C:\Users\KOlso\Documents\ArcGIS\Projects\5571_Final_Proj\all_scores.csv,time_8mph_lowq,-1,-1;time_12mph_lowq "time_12mph_lowq" true true false 4 Long 0 0,First,#,C:\Users\KOlso\Documents\ArcGIS\Projects\5571_Final_Proj\all_scores.csv,time_12mph_lowq,-1,-1;time_18mph_lowq "time_18mph_lowq" true true false 4 Long 0 0,First,#,C:\Users\KOlso\Documents\ArcGIS\Projects\5571_Final_Proj\all_scores.csv,time_18mph_lowq,-1,-1;baseline_lowq "baseline_lowq" true true false 4 Long 0 0,First,#,C:\Users\KOlso\Documents\ArcGIS\Projects\5571_Final_Proj\all_scores.csv,baseline_lowq,-1,-1;delta_lowq "delta_lowq" true true false 4 Long 0 0,First,#,C:\Users\KOlso\Documents\ArcGIS\Projects\5571_Final_Proj\all_scores.csv,delta_lowq,-1,-1;overall_range "overall_range" true true false 4 Long 0 0,First,#,C:\Users\KOlso\Documents\ArcGIS\Projects\5571_Final_Proj\all_scores.csv,overall_range,-1,-1',
    sort_field=None
)

<Result 'C:\\Users\\KOlso\\Documents\\ArcGIS\\Projects\\5571_Final_Proj\\5571_Final_Proj.gdb\\all_scores'>

In [ ]:
#join scores to points
arcpy.management.AddJoin(
    in_layer_or_view="bike_points_100",
    in_field="Point_ID",
    join_table="all_scores",
    join_field="Field1",
    join_type="KEEP_ALL",
    index_join_fields="NO_INDEX_JOIN_FIELDS",
    rebuild_index="NO_REBUILD_INDEX",
    join_operation=""
)

#export combined table
arcpy.conversion.ExportFeatures(
    in_features="bike_points_100",
    out_features=r"C:\Users\KOlso\Documents\ArcGIS\Projects\5571_Final_Proj\5571_Final_Proj.gdb\bike_points_scored",
    where_clause="",
    use_field_alias_as_name="NOT_USE_ALIAS",
    field_mapping='CID "CID" true true false 4 Long 0 0,First,#,bike_points_100,bike_points_100.CID,-1,-1;Point_ID "Point_ID" true true false 4 Long 0 0,First,#,bike_points_100,bike_points_100.Point_ID,-1,-1;OBJECTID "OBJECTID" false true false 4 Long 0 9,First,#,bike_points_100,all_scores.OBJECTID,-1,-1;Field1 "Field1" true true false 4 Long 0 0,First,#,bike_points_100,all_scores.Field1,-1,-1;time_8mph_highq "time_8mph_highq" true true false 4 Long 0 0,First,#,bike_points_100,all_scores.time_8mph_highq,-1,-1;time_12mph_highq "time_12mph_highq" true true false 4 Long 0 0,First,#,bike_points_100,all_scores.time_12mph_highq,-1,-1;time_18mph_highq "time_18mph_highq" true true false 4 Long 0 0,First,#,bike_points_100,all_scores.time_18mph_highq,-1,-1;baseline_highq "baseline_highq" true true false 4 Long 0 0,First,#,bike_points_100,all_scores.baseline_highq,-1,-1;delta_highq "delta_highq" true true false 4 Long 0 0,First,#,bike_points_100,all_scores.delta_highq,-1,-1;time_8mph_medq "time_8mph_medq" true true false 4 Long 0 0,First,#,bike_points_100,all_scores.time_8mph_medq,-1,-1;time_12mph_medq "time_12mph_medq" true true false 4 Long 0 0,First,#,bike_points_100,all_scores.time_12mph_medq,-1,-1;time_18mph_medq "time_18mph_medq" true true false 4 Long 0 0,First,#,bike_points_100,all_scores.time_18mph_medq,-1,-1;baseline_medq "baseline_medq" true true false 4 Long 0 0,First,#,bike_points_100,all_scores.baseline_medq,-1,-1;delta_medq "delta_medq" true true false 4 Long 0 0,First,#,bike_points_100,all_scores.delta_medq,-1,-1;time_8mph_lowq "time_8mph_lowq" true true false 4 Long 0 0,First,#,bike_points_100,all_scores.time_8mph_lowq,-1,-1;time_12mph_lowq "time_12mph_lowq" true true false 4 Long 0 0,First,#,bike_points_100,all_scores.time_12mph_lowq,-1,-1;time_18mph_lowq "time_18mph_lowq" true true false 4 Long 0 0,First,#,bike_points_100,all_scores.time_18mph_lowq,-1,-1;baseline_lowq "baseline_lowq" true true false 4 Long 0 0,First,#,bike_points_100,all_scores.baseline_lowq,-1,-1;delta_lowq "delta_lowq" true true false 4 Long 0 0,First,#,bike_points_100,all_scores.delta_lowq,-1,-1;overall_range "overall_range" true true false 4 Long 0 0,First,#,bike_points_100,all_scores.overall_range,-1,-1',
    sort_field=None
)

In [56]:
#replace all null scores with 0 - if we can't give it a score, it's 0
#solution found: https://support.esri.com/en-us/knowledge-base/how-to-replace-null-values-with-zeroes-in-an-attribute--000023190

path = r'C:\Users\KOlso\Documents\ArcGIS\Projects\5571_Final_Proj\5571_Final_Proj.gdb\bike_points_scored'
fieldObs = arcpy.ListFields(path)  
fieldNames = []  
for field in fieldObs:  
    fieldNames.append(field.name)  
del fieldObs  
fieldCount = len(fieldNames)  

with arcpy.da.UpdateCursor(path, fieldNames) as curU:  
    for row in curU:  
        rowU = row  
        for field in range(fieldCount):  
            if rowU[field] == None:  
                rowU[field] = "0"  
      
      
        curU.updateRow(rowU)

del curU

#May need to remove table and re-add it

In [14]:
merged_df = pd.concat([low_qual_df, med_qual_df, high_qual_df], ignore_index=True)

merged_df

,Name,OriginID,DestinationID,DestinationRank,Total_Length,Shape_Length,time_8mph,time_12mph,time_18mph,network_quality
0,1 - 68,1,68,2,1541.157223,1170.750337,430.934887,287.289924,191.526616,low-qual
1,1 - 88,1,88,3,4681.802789,2874.576249,1309.115075,872.743383,581.828922,low-qual
2,1 - 76,1,76,4,6786.820728,4031.345475,1897.715416,1265.143611,843.429074,low-qual
3,1 - 80,1,80,5,7730.412806,6349.349078,2161.560492,1441.040328,960.693552,low-qual
4,1 - 62,1,62,6,11487.428758,6369.520032,3212.088770,2141.392513,1427.595009,low-qual
...,...,...,...,...,...,...,...,...,...,...
16387,99 - 65,99,65,40,65397.193277,32316.303367,18286.214830,12190.809887,8127.206591,high-qual
16388,99 - 78,99,78,41,66475.501192,34304.336472,18587.728843,12391.819229,8261.212819,high-qual
16389,99 - 69,99,69,42,68679.022817,35938.427557,19203.872561,12802.581708,8535.054472,high-qual
16390,99 - 20,99,20,43,69430.987820,36374.956668,19414.135310,12942.756873,8628.504582,high-qual


In [13]:
melted_df = pd.melt(
    merged_df, 
    id_vars = ["Name", "OriginID", "DestinationID", "Total_Length", "network_quality"],
    value_vars = ["time_8mph", "time_12mph", "time_18mph"],
    var_name = "speed",
    value_name = "time_secs"
)

NameError: name 'merged_df' is not defined

In [18]:
melted_df

,Name,OriginID,DestinationID,Total_Length,network_quality,speed,time_secs
0,1 - 68,1,68,1541.157223,low-qual,time_8mph,430.934887
1,1 - 88,1,88,4681.802789,low-qual,time_8mph,1309.115075
2,1 - 76,1,76,6786.820728,low-qual,time_8mph,1897.715416
3,1 - 80,1,80,7730.412806,low-qual,time_8mph,2161.560492
4,1 - 62,1,62,11487.428758,low-qual,time_8mph,3212.088770
...,...,...,...,...,...,...,...
49171,99 - 65,99,65,65397.193277,high-qual,time_18mph,8127.206591
49172,99 - 78,99,78,66475.501192,high-qual,time_18mph,8261.212819
49173,99 - 69,99,69,68679.022817,high-qual,time_18mph,8535.054472
49174,99 - 20,99,20,69430.987820,high-qual,time_18mph,8628.504582


In [20]:
sns.boxplot(data=melted_df, x = "network_quality", y = "time_secs", hue = "speed")
plt.show()

In [115]:
stats = (
    melted_df
    .groupby(['quality_label', 'speed'])['time_secs']
    .agg(
        lower_quartile=lambda x: x.quantile(0.25),
        median='median',
        upper_quartile=lambda x: x.quantile(0.75),
        min='min',
        max='max'
    )
    .reset_index()
)

# Print the statistics
stats

,quality_label,speed,lower_quartile,median,upper_quartile,min,max
0,high-qual,time_12mph,3097.226626,5312.326338,8035.748948,18.741541,13475.045348
1,high-qual,time_18mph,2064.817751,3541.550892,5357.165966,12.494360,8983.363565
2,high-qual,time_8mph,4645.839940,7968.489507,12053.623423,28.112311,20212.568022
3,low-qual,time_12mph,2910.653349,4403.582155,6098.145963,209.502992,11340.013281
4,low-qual,time_18mph,1940.435566,2935.721437,4065.430642,139.668661,7560.008854
5,low-qual,time_8mph,4365.980023,6605.373232,9147.218944,314.254488,17010.019921
6,med-qual,time_12mph,3939.284677,6091.621331,8579.117575,18.741538,15548.274599
7,med-qual,time_18mph,2626.189785,4061.080887,5719.411717,12.494359,10365.516399
8,med-qual,time_8mph,5908.927016,9137.431996,12868.676363,28.112307,23322.411899


In [15]:
# Find common values in the 'Name' field across all three DataFrames
common_names = set(low_qual_df['Name']).intersection(med_qual_df['Name']).intersection(high_qual_df['Name'])

# Filter each DataFrame to only include rows with the common 'Name' values
low_qual_filtered = low_qual_df[low_qual_df['Name'].isin(common_names)]
med_qual_filtered = med_qual_df[med_qual_df['Name'].isin(common_names)]
high_qual_filtered = high_qual_df[high_qual_df['Name'].isin(common_names)]

# Concatenate the filtered DataFrames
merged_common_df = pd.concat([low_qual_filtered, med_qual_filtered, high_qual_filtered], ignore_index=True)

# Display the resulting DataFrame
merged_common_df


,Name,OriginID,DestinationID,DestinationRank,Total_Length,Shape_Length,time_8mph,time_12mph,time_18mph,network_quality
0,1 - 68,1,68,2,1541.157223,1170.750337,430.934887,287.289924,191.526616,low-qual
1,1 - 88,1,88,3,4681.802789,2874.576249,1309.115075,872.743383,581.828922,low-qual
2,1 - 80,1,80,5,7730.412806,6349.349078,2161.560492,1441.040328,960.693552,low-qual
3,1 - 47,1,47,10,15471.416256,13707.620135,4326.082316,2884.054878,1922.703252,low-qual
4,2 - 86,2,86,2,1565.454248,1032.522242,437.728766,291.819177,194.546118,low-qual
...,...,...,...,...,...,...,...,...,...,...
5281,99 - 65,99,65,40,65397.193277,32316.303367,18286.214830,12190.809887,8127.206591,high-qual
5282,99 - 78,99,78,41,66475.501192,34304.336472,18587.728843,12391.819229,8261.212819,high-qual
5283,99 - 69,99,69,42,68679.022817,35938.427557,19203.872561,12802.581708,8535.054472,high-qual
5284,99 - 20,99,20,43,69430.987820,36374.956668,19414.135310,12942.756873,8628.504582,high-qual


In [16]:
common_melted_df = pd.melt(
    merged_common_df, 
    id_vars = ["Name", "OriginID", "DestinationID", "Total_Length", "network_quality"],
    value_vars = ["time_8mph", "time_12mph", "time_18mph"],
    var_name = "speed",
    value_name = "time_secs"
)

In [17]:
sns.boxplot(data=common_melted_df, x = "network_quality", y = "time_secs", hue = "speed")
plt.show()

In [19]:
stats_comm = (
    common_melted_df
    .groupby(['network_quality', 'speed'])['time_secs']
    .agg(
        lower_quartile=lambda x: x.quantile(0.25),
        median='median',
        upper_quartile=lambda x: x.quantile(0.75),
        min='min',
        max='max'
    )
    .reset_index()
)

# Print the statistics
stats_comm

,network_quality,speed,lower_quartile,median,upper_quartile,min,max
0,high-qual,time_12mph,3069.238360,5286.425560,8013.286807,18.741541,13475.045348
1,high-qual,time_18mph,2046.158906,3524.283707,5342.191204,12.494360,8983.363565
2,high-qual,time_8mph,4603.857539,7929.638340,12019.930210,28.112311,20212.568022
3,low-qual,time_12mph,2275.187610,3538.837104,5000.357901,209.502992,9324.933715
4,low-qual,time_18mph,1516.791740,2359.224736,3333.571934,139.668661,6216.622477
5,low-qual,time_8mph,3412.781416,5308.255656,7500.536851,314.254488,13987.400572
6,med-qual,time_12mph,2967.392024,4663.829102,6995.056995,18.741538,12643.134803
7,med-qual,time_18mph,1978.261349,3109.219401,4663.371330,12.494359,8428.756535
8,med-qual,time_8mph,4451.088036,6995.743652,10492.585492,28.112307,18964.702205
